In [1]:
import pandas as pd
import h2o
from h2o.automl import H2OAutoML

In [3]:
train = pd.read_csv('./output/f59.csv')
test = pd.read_csv('./output/f20.csv')
testid = test['sofifa_id']

In [4]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,1 hour 18 mins
H2O cluster timezone:,Europe/Madrid
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.1.1
H2O cluster version age:,2 days
H2O cluster name:,H2O_from_python_jose_lbwmba
H2O cluster total nodes:,1
H2O cluster free memory:,1.232 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


In [5]:
h2train = h2o.H2OFrame(train)
h2test = h2o.H2OFrame(test)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [6]:
columnas = [a for a in h2train.columns if a != 'overall']
x = columnas
y = 'overall'
print(columnas)

['sofifa_id', 'age', 'height_cm', 'weight_kg', 'nationality', 'club', 'potential', 'value_eur', 'wage_eur', 'preferred_foot', 'international_reputation', 'weak_foot', 'skill_moves', 'work_rate', 'pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic', 'gk_diving', 'gk_handling', 'gk_kicking', 'gk_reflexes', 'gk_speed', 'gk_positioning', 'attacking_crossing', 'attacking_finishing', 'attacking_heading_accuracy', 'attacking_short_passing', 'attacking_volleys', 'skill_dribbling', 'skill_curve', 'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control', 'movement_acceleration', 'movement_sprint_speed', 'movement_agility', 'movement_reactions', 'movement_balance', 'power_shot_power', 'power_jumping', 'power_stamina', 'power_strength', 'power_long_shots', 'mentality_aggression', 'mentality_interceptions', 'mentality_positioning', 'mentality_vision', 'mentality_penalties', 'defending_marking', 'defending_standing_tackle', 'defending_sliding_tackle', 'goalkeeping_diving', 'goalk

H2oAutoML buscará automáticamente modelos de machine learning que se ajustan a los datos y entrenaremos con el mejor

In [7]:
automl = H2OAutoML(max_models=50, seed=42,max_runtime_secs=300, sort_metric='RMSE')
automl.train(x=x, y=y, training_frame=h2train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [8]:
print('[INFO] Models leader board:')
leader_board = automl.leaderboard
leader_board.head()

[INFO] Models leader board:


model_id,rmse,mean_residual_deviance,mse,mae,rmsle
XGBoost_1_AutoML_20200308_215025,0.568767,0.323496,0.323496,0.381494,0.00926448
StackedEnsemble_AllModels_AutoML_20200308_215025,0.569952,0.324845,0.324845,0.381686,0.00928626
XGBoost_2_AutoML_20200308_215025,26.6447,709.94,709.94,25.7847,0.514139


In [9]:
metrics = automl.leader.cross_validation_metrics_summary().as_data_frame()
metrics

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,mae,0.38149366,0.015364289,0.38895828,0.3896793,0.3832404,0.39105904,0.35453123
1,mean_residual_deviance,0.32349584,0.022638157,0.3269496,0.3390068,0.32436463,0.34186324,0.28529492
2,mse,0.32349584,0.022638157,0.3269496,0.3390068,0.32436463,0.34186324,0.28529492
3,r2,0.99364567,4.4327098E-4,0.99358207,0.9933429,0.99362856,0.9932822,0.9943926
4,residual_deviance,0.32349584,0.022638157,0.3269496,0.3390068,0.32436463,0.34186324,0.28529492
5,rmse,0.5684778,0.020274466,0.57179505,0.5822429,0.5695302,0.5846907,0.53413004
6,rmsle,0.009259799,3.2925355E-4,0.0093016755,0.009462125,0.009258627,0.009563377,0.00871319


In [10]:
automl.leader.varimp(use_pandas=True)

,variable,relative_importance,scaled_importance,percentage
0,value_eur,9.630670e+06,1.000000,0.397986
1,movement_reactions,6.088596e+06,0.632209,0.251611
2,potential,1.966108e+06,0.204151,0.081249
3,age,1.396309e+06,0.144986,0.057702
4,wage_eur,8.232186e+05,0.085479,0.034019
...,...,...,...,...
81,work_rate,3.387718e+02,0.000035,0.000014
82,skill_moves,2.551435e+02,0.000026,0.000011
83,weak_foot,2.542814e+02,0.000026,0.000011
84,preferred_foot,2.433620e+02,0.000025,0.000010


In [11]:
predictions = automl.leader.predict(h2test)

xgboost prediction progress: |████████████████████████████████████████████| 100%


In [13]:
test['overall'] = predictions.as_data_frame()
columnasentrega = ['sofifa_id','overall']
entrega = test['overall']
test = test[columnasentrega]
test

,sofifa_id,overall
0,158023,93.334595
1,20801,92.799469
2,190871,92.543037
3,200389,90.172699
4,183277,91.046577
...,...,...
18273,245006,49.266388
18274,250995,49.542862
18275,252332,49.423573
18276,251110,48.024727


In [14]:
test.to_csv('output/predictfinal2.csv', index=False)

In [16]:
testing = pd.read_csv('output/predictfinal2.csv')

In [17]:
h2o.cluster().shutdown()

H2O session _sid_ab15 closed.


**XGBoost_1_AutoML_20200308_215025**


Este es el modelo que da mejores resultados